# GRASS Tool with Parameters

## Interface Definition for Scripts

Since GRASS tools are executable scripts (or generally programs), the interface of a GRASS tool is the command line interface of a Python script. A dedicated function _grass.script.parser_ takes care of processing the command line arguments based on the interface description specified in a Python comment with a key-value syntax defined by GRASS GIS.

The following is an example of a script which takes two parameters: name of a vector map and name of a raster map:

In [40]:
%%writefile vector_to_raster.py
#!/usr/bin/env python

#%module
#% description: Compute viewshed and compute statistics about visible parts of sample layers
#% keyword: viewshed
#% keyword: geometry
#%end
#%option G_OPT_R_ELEV
#% description: Name of input elevation raster map
#%end
#%option G_OPT_V_INPUT
# % key: points
#%end
#%option G_OPT_R_INPUTS
#%end
#%option G_OPT_F_OUTPUT
# % guisection: Output
#%end
# %option
# % key: format
# % type: string
# % required: yes
# % options: json,csv
# % label: Output format
# % descriptions: json;JSON (JavaScript Object Notation);csv;CSV (Comma Separated Values)
# % answer: json
# % guisection: Output
# %end


import atexit
import subprocess
import multiprocessing
import sys
import csv
import json
import io
import tempfile

from pathlib import Path

import grass.script as gs

def output_results(results, sample_rasters):
    with tempfile.TemporaryDirectory() as tmp_dir:
        file_name = Path(tmp_dir) / "data.txt"
        header = []
        for name in sample_rasters:
            for value in ["mean", "stddev", "min", "max"]:
                header.append(f"{name}_{value}")
        with open(file_name, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=header)
            writer.writeheader()
            for point, result in results:
                row = {}
                row.update(point)
                for raster_name, statistics in result:
                    for key, value in statistics:
                        new_key = f"{raster_name}_{key}"
                        point[new_key] = value
                writer.writerow(row)
                print(row)
        gs.run_command("v.in.ascii", input=file_name, format="point", separator="comma", flags="cr")

def clean(name):
    gs.run_command("g.remove", type="raster", name=name, flags="f", superquiet=True)

def multiple_viewsheds(
    elevation,
    points,
    sample_rasters,
    output,
    file_format,
):
    point_data = gs.read_command("v.out.ascii", input=points, type="point", format="point", separator="comma", flags="cr")
    reader = csv.DictReader(io.StringIO(point_data))  
    data = []
    for point in reader:
        
        data.append( (elevation,
            point,
            sample_rasters,
            output,
            file_format))
     
    nprocs = 12
    with multiprocessing.Pool(processes=nprocs) as pool:
        #results = pool.map_async(parametrized_viewshed, data).get()
        results = pool.starmap(parametrized_viewshed, data)
    
    
    output_results(results, sample_rasters)

        
def parametrized_viewshed(elevation,
        point,
        sample_rasters,
        output,
        file_format):
    coordinates = (point["east"], point["north"])
    result = one_viewshed(
        elevation=elevation,
        coordinates=coordinates,
        sample_rasters=sample_rasters,
        output=output,
        file_format=file_format,
    )
    return point, result
    
def one_viewshed(
    elevation,
    coordinates,
    sample_rasters,
    output,
    file_format,
):
    viewshed = gs.append_node_pid(gs.legalize_vector_name(f"tmp_viewshed_{coordinates}"))
    atexit.register(clean, viewshed)
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=viewshed,
        coordinates=coordinates,
        flags="cb",
        quiet=True,
    )
    gs.run_command("r.null", map=viewshed, setnull=0)
    results = {}
    for name in sample_rasters:
        table_data = gs.read_command(
                "r.univar",
                map=name,
                zones=viewshed,
                quiet=True,
                flags="t",
                separator="comma",
        )
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            del row["zone"]
            del row["label"]
            del row["non_null_cells"]
            del row["null_cells"]
            results[name] = row
    return results


def main():
    options, flags = gs.parser()
    sample_rasters = options["input"].split(",")
    multiple_viewsheds(
        elevation=options["elevation"],
        points = options["points"],
        sample_rasters=sample_rasters,
        output=options["output"],
        file_format=options["format"],
    )


if __name__ == "__main__":
    main()

Overwriting vector_to_raster.py


As before, we will make the script executable:

In [41]:
!chmod u+x vector_to_raster.py

Running the script with `--help` gives its interface described for command line use:

In [42]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./vector_to_raster.py elevation=elevation points=firestations input=elevation,ndvi output="data.txt" format="csv" --o
#!cat data.txt

Starting GRASS GIS...
Cleaning up temporary files...
Executing <./vector_to_raster.py elevation=elevation points=firestations input=elevation,ndvi output=data.txt format=csv --o> ...
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50   0  52   2  54   4  56   6   8  58  10  60  12  62  14  64  16  18  66  20  68  22  24  26  70  28  72  30  74  32  34  76  36  78  38  80  40  82  42  84  44  86  46  48  50  88  52  90  54  92  56  94  58  96  60  98  62 100
  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
Traceback (most recent call last):
  File "./vector_to_raster.py", line 161, in <module>
    main()
  File "./vector_to_raster.py", line 151, in main
    multiple_viewsheds(
  File "./vector_to_raster.p

In [3]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec i.vi red=lsat7_2002_30 nir=lsat7_2002_40 viname=ndvi output=ndvi

Starting GRASS GIS...
Cleaning up temporary files...
Executing <i.vi red=lsat7_2002_30 nir=lsat7_2002_40 viname=ndvi output=ndvi> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Execution of <i.vi red=lsat7_2002_30 nir=lsat7_2002_40 viname=ndvi output=ndvi> finished.
Cleaning up temporary files...


Running the script with `--html-description` gives the command line interface described in HTML which later becomes a part of the tool's HTML documentation:

In [51]:
!grass ~/grassdata/nc_spm_grass7/foss4g --exec r.univar map=elevation zones=viewshed -t separator=comma

Starting GRASS GIS...
Cleaning up temporary files...
Executing <r.univar map=elevation zones=viewshed -t separator=comma> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
zone,label,non_null_cells,null_cells,min,max,range,mean,mean_of_abs,stddev,variance,coeff_var,sum,sum_abs
1,,121,0,65.0054626464844,152.262680053711,87.2572174072266,115.819097786895,115.819097786895,20.486681456427,419.704117097109,17.6885175656626,14014.1108322144,14014.1108322144
Execution of <r.univar map=elevation zones=viewshed -t separator=comma> finished.
Cleaning up temporary files...


In [27]:
!black vector_to_raster.py
!flake8 vector_to_raster.py
!pylint vector_to_raster.py

reformatted vector_to_raster.py

All done! ✨ 🍰 ✨
1 file reformatted.
vector_to_raster.py:4:80: E501 line too long (93 > 79 characters)
vector_to_raster.py:25:80: E501 line too long (89 > 79 characters)
vector_to_raster.py:32:1: F401 'subprocess' imported but unused
vector_to_raster.py:34:1: F401 'sys' imported but unused
vector_to_raster.py:61:80: E501 line too long (84 > 79 characters)
vector_to_raster.py:95:80: E501 line too long (81 > 79 characters)
************* Module vector_to_raster
vector_to_raster.py:1:0: C0114: Missing module docstring (missing-module-docstring)
vector_to_raster.py:39:0: E0401: Unable to import 'grass.script' (import-error)
vector_to_raster.py:42:0: C0116: Missing function or method docstring (missing-function-docstring)
vector_to_raster.py:60:0: C0116: Missing function or method docstring (missing-function-docstring)
vector_to_raster.py:64:0: C0116: Missing function or method docstring (missing-function-docstring)
vector_to_raster.py:80:13: R1734: Consider u

# Points


In [66]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec g.region raster=elevation
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec v.out.ascii input=firestations type="point" format="point" separator="comma" -c -r

Starting GRASS GIS...
Cleaning up temporary files...
Executing <g.region raster=elevation> ...
Execution of <g.region raster=elevation> finished.
Cleaning up temporary files...
Starting GRASS GIS...
Cleaning up temporary files...
Executing <v.out.ascii input=firestations type=point format=point separator=comma -c -r> ...
east,north,cat
630420.50034711,215694.02807559,9
630879.21198056,224876.55413017,12
635775.56533925,228121.69258378,19
635940.26230542,225912.79645818,20
637386.83129147,222569.15159736,21
641437.43097947,221737.70843723,22
644588.46729229,217450.61375119,23
644598.62057612,221014.97865205,24
642868.29195637,225195.28915104,25
640173.91152612,226162.37987726,26
640711.47282426,228461.46577181,27
644601.8806259,226660.12097034,28
633178.15477,221353.03723179,52
641803.17360861,224986.14236578,56
642559.95242121,215531.80353493,57
Execution of <v.out.ascii input=firestations type=point format=point separator=comma -c -r> finished.
Cleaning up temporary files...
